In [ ]:
# Rebuild regional merge (MDY parsing) and save with dd/mm/yyyy formatting
!pip -q install XlsxWriter

from google.colab import drive, files
drive.mount('/content/drive')

import pandas as pd
from pathlib import Path
from functools import reduce

BASE = Path("/content/drive/MyDrive/Datasets")        # <- your folder
OUT  = BASE / "MERGED_OUTPUTS"; OUT.mkdir(parents=True, exist_ok=True)

# --- find files by fragments (case-insensitive)
def find_one(*frags):
    frags = [f.lower() for f in frags]
    for p in BASE.iterdir():
        if p.is_file() and all(f in p.stem.lower() for f in frags):
            return p
    raise FileNotFoundError(f"Missing file with fragments: {frags}")

paths = {
    "HPI"      : find_one("hpi_clean_panel"),
    "AWE"      : find_one("averageearnings_monthly_cleaned_2005_2025"),
    "UNEMP"    : find_one("unemploymentrate_monthly_cleaned_2005_2025"),
    "CPI"      : find_one("cpi_monthly_cleaned_2005_2025"),
    "APPR"     : find_one("mortgageapprovals_monthly_cleaned_2005_2025"),
    "MORT2Y"   : find_one("mortgagerate_monthly_cleaned_2005_2025"),
    "BANKRATE" : find_one("bankrate_monthly_cleaned_2005_2025"),
    "CONF"     : find_one("ConsumerConfidence_monthly_cleaned_2005_2025_modified"),
    "CONST"    : find_one("BuildingMaterials_monthly_cleaned_2005_2025"),
    "TRENDS"   : find_one("trends_long_monthly_new"),
}

# --- helpers
def read_any(p: Path):
    if p.suffix.lower() in (".xlsx",".xls"): return pd.read_excel(p)
    try: return pd.read_csv(p)
    except UnicodeDecodeError: return pd.read_csv(p, encoding="latin1")

def to_month_mdy(s):
    # FIX: use how="start" (not "start" positional)
    return pd.to_datetime(s, errors="coerce", dayfirst=False).dt.to_period("M").dt.to_timestamp(how="start")

def clean_numeric(df):
    for c in df.columns:
        if c == "Date": continue
        df[c] = (df[c].astype(str)
                        .str.replace("%","", regex=False)
                        .str.replace(",","", regex=False)
                        .str.extract(r"([-+]?\d*\.?\d+)")[0]
                        .astype(float))
    return df

def load_macro_mdy(path, rename=None, prefix=None):
    df = read_any(path).copy()
    date_col = "Date" if "Date" in df.columns else df.columns[0]
    df["Date"] = to_month_mdy(df[date_col])
    df = df.dropna(subset=["Date"])
    df = df.drop(columns=[c for c in df.columns if c.lower() in ("year","month") and c!="Date"], errors="ignore")
    df = df[["Date"] + [c for c in df.columns if c!="Date"]]
    df = clean_numeric(df)
    if rename: df = df.rename(columns=rename)
    if prefix: df = df.rename(columns={c: f"{prefix}{c}" for c in df.columns if c!="Date"})
    return (df.sort_values("Date")
              .groupby("Date", as_index=False)
              .agg({c:"last" for c in df.columns if c!="Date"}))

# --- load macros (all MDY)
awe   = load_macro_mdy(paths["AWE"],   {"TotalWeeklyEarnings":"AWE_Total","RegularWeeklyEarnings":"AWE_Regular",
                                        "awe_total":"AWE_Total","awe_regular":"AWE_Regular"})
unemp = load_macro_mdy(paths["UNEMP"], {"UnemploymentRate":"UnemploymentRate"})
cpi   = load_macro_mdy(paths["CPI"],   {"CPI":"CPI"})
appr  = load_macro_mdy(paths["APPR"],  {"MortgageApprovals":"MortgageApprovals"})
mort2 = load_macro_mdy(paths["MORT2Y"],{"MortgageRate":"MortgageRate_2YFix"})
br    = load_macro_mdy(paths["BANKRATE"],{"BankRate":"BankRate"})
conf  = load_macro_mdy(paths["CONF"],  {"ConsumerConfidence":"ConsumerConfidence"})
const = load_macro_mdy(paths["CONST"], prefix="BM_")
trends= load_macro_mdy(paths["TRENDS"])
trends = trends.rename(columns={c: (c if str(c).lower().startswith("gt_") else f"gt_{c}") for c in trends.columns if c!="Date"})

macro = reduce(lambda l,r: pd.merge(l, r, on="Date", how="outer"),
               [awe, unemp, cpi, appr, mort2, br, conf, const, trends])

# --- load HPI (explicit MDY)
hpi_raw = read_any(paths["HPI"]).copy()
if "Date" not in hpi_raw.columns:
    raise ValueError("HPI file must have a 'Date' column.")
hpi_raw["Date"] = to_month_mdy(hpi_raw["Date"])

# normalise identifiers
for c in list(hpi_raw.columns):
    cl = c.lower()
    if "region" in cl and "name" in cl and "RegionName" not in hpi_raw.columns:
        hpi_raw = hpi_raw.rename(columns={c:"RegionName"})
    if cl in ("areacode","area_code","code") and "AreaCode" not in hpi_raw.columns:
        hpi_raw = hpi_raw.rename(columns={c:"AreaCode"})

keep = [c for c in ["Date","RegionName","AreaCode","AveragePrice","Index","SalesVolume"] if c in hpi_raw.columns]
hpi   = hpi_raw[keep].sort_values(["RegionName","Date"] if "RegionName" in keep else "Date")

# --- clip to study window and merge
start, end = pd.Timestamp(2005,1,1), pd.Timestamp(2025,6,1)
macro = macro[(macro["Date"]>=start) & (macro["Date"]<=end)]
hpi   = hpi[(hpi["Date"]>=start) & (hpi["Date"]<=end)]

hpi_reg = pd.merge(hpi, macro, on="Date", how="left", validate="m:1")

# --- save with dd/mm/yyyy formatting
hpi_reg = hpi_reg.copy()
hpi_reg["Date"] = pd.to_datetime(hpi_reg["Date"])

xlsx_path = OUT / "HPI_regional_merged_2005_2025_ddmmyyyy_MODIFIED.xlsx"
with pd.ExcelWriter(xlsx_path, engine="xlsxwriter", datetime_format="dd/mm/yyyy") as xl:
    hpi_reg.to_excel(xl, index=False, sheet_name="HPI_Regional")
    ws = xl.sheets["HPI_Regional"]; wb = xl.book
    ws.set_column(hpi_reg.columns.get_loc("Date"), hpi_reg.columns.get_loc("Date"), 12,
                  wb.add_format({"num_format":"dd/mm/yyyy"}))

csv_path = OUT / "HPI_regional_merged_2005_2025_ddmmyyyy_MODIFIED.csv"
tmp = hpi_reg.copy(); tmp["Date"] = tmp["Date"].dt.strftime("%d/%m/%Y"); tmp.to_csv(csv_path, index=False)

print("Saved Excel:", xlsx_path)
print("Saved CSV  :", csv_path)
files.download(str(xlsx_path))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.4 MB/s eta 0:00:00
Mounted at /content/drive


/tmp/ipython-input-2913269189.py:43: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(s, errors="coerce", dayfirst=False).dt.to_period("M").dt.to_timestamp(how="start")


Saved Excel: /content/drive/MyDrive/Datasets/MERGED_OUTPUTS/HPI_regional_merged_2005_2025_ddmmyyyy_MODIFIED.xlsx
Saved CSV  : /content/drive/MyDrive/Datasets/MERGED_OUTPUTS/HPI_regional_merged_2005_2025_ddmmyyyy_MODIFIED.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ============================================================
# Build regional HPI–macro–GT keywords merge (MDY date parsing)
# ============================================================
!pip -q install XlsxWriter

from google.colab import drive, files
drive.mount('/content/drive')

import pandas as pd
from pathlib import Path
from functools import reduce

BASE = Path("/content/drive/MyDrive/Datasets")        # <- your folder
OUT  = BASE / "MERGED_OUTPUTS"; OUT.mkdir(parents=True, exist_ok=True)

# --- find files by fragments (case-insensitive)
def find_one(*frags):
    frags = [f.lower() for f in frags]
    for p in BASE.iterdir():
        if p.is_file() and all(f in p.stem.lower() for f in frags):
            return p
    raise FileNotFoundError(f"Missing file with fragments: {frags}")

paths = {
    "HPI"      : find_one("hpi_clean_panel"),
    "AWE"      : find_one("averageearnings_monthly_cleaned_2005_2025"),
    "UNEMP"    : find_one("unemploymentrate_monthly_cleaned_2005_2025"),
    "CPI"      : find_one("cpi_monthly_cleaned_2005_2025"),
    "APPR"     : find_one("mortgageapprovals_monthly_cleaned_2005_2025"),
    "MORT2Y"   : find_one("mortgagerate_monthly_cleaned_2005_2025"),
    "BANKRATE" : find_one("bankrate_monthly_cleaned_2005_2025"),
    "CONF"     : find_one("ConsumerConfidence_monthly_cleaned_2005_2025_modified"),
    "CONST"    : find_one("BuildingMaterials_monthly_cleaned_2005_2025"),
    "TRENDS"   : find_one("trends_long_monthly_new"),
}

# --- helpers
def read_any(p: Path):
    if p.suffix.lower() in (".xlsx",".xls"):
        return pd.read_excel(p)
    try:
        return pd.read_csv(p)
    except UnicodeDecodeError:
        return pd.read_csv(p, encoding="latin1")

def to_month_mdy(s):
    # parse MDY, convert to month start
    return pd.to_datetime(s, errors="coerce", dayfirst=False).dt.to_period("M").dt.to_timestamp(how="start")

def clean_numeric(df):
    for c in df.columns:
        if c == "Date":
            continue
        df[c] = (
            df[c].astype(str)
                 .str.replace("%","", regex=False)
                 .str.replace(",","", regex=False)
                 .str.extract(r"([-+]?\d*\.?\d+)")[0]
                 .astype(float)
        )
    return df

def load_macro_mdy(path, rename=None, prefix=None):
    df = read_any(path).copy()
    date_col = "Date" if "Date" in df.columns else df.columns[0]
    df["Date"] = to_month_mdy(df[date_col])
    df = df.dropna(subset=["Date"])
    df = df.drop(columns=[c for c in df.columns if c.lower() in ("year","month") and c!="Date"], errors="ignore")
    df = df[["Date"] + [c for c in df.columns if c!="Date"]]
    df = clean_numeric(df)
    if rename:
        df = df.rename(columns=rename)
    if prefix:
        df = df.rename(columns={c: f"{prefix}{c}" for c in df.columns if c!="Date"})
    return (
        df.sort_values("Date")
          .groupby("Date", as_index=False)
          .agg({c:"last" for c in df.columns if c!="Date"})
    )

# --- load macro series (all MDY)
awe   = load_macro_mdy(paths["AWE"],   {"TotalWeeklyEarnings":"AWE_Total","RegularWeeklyEarnings":"AWE_Regular",
                                        "awe_total":"AWE_Total","awe_regular":"AWE_Regular"})
unemp = load_macro_mdy(paths["UNEMP"], {"UnemploymentRate":"UnemploymentRate"})
cpi   = load_macro_mdy(paths["CPI"],   {"CPI":"CPI"})
appr  = load_macro_mdy(paths["APPR"],  {"MortgageApprovals":"MortgageApprovals"})
mort2 = load_macro_mdy(paths["MORT2Y"],{"MortgageRate":"MortgageRate_2YFix"})
br    = load_macro_mdy(paths["BANKRATE"],{"BankRate":"BankRate"})
conf  = load_macro_mdy(paths["CONF"],  {"ConsumerConfidence":"ConsumerConfidence"})
const = load_macro_mdy(paths["CONST"], prefix="BM_")

# --- load Google Trends keywords (long → wide)
tr_raw = read_any(paths["TRENDS"]).copy()
# Expect columns: keyword, theme, date, hits
tr_raw["Date"] = to_month_mdy(tr_raw["date"])
tr_raw = tr_raw.dropna(subset=["Date"])

# build safe column names: gt_{theme}__{clean_keyword}
tr_raw["keyword_clean"] = (
    tr_raw["keyword"]
        .astype(str)
        .str.lower()
        .str.strip()
        .str.replace(r"[^0-9a-z]+", "_", regex=True)
        .str.strip("_")
)
tr_raw["colname"] = "gt_" + tr_raw["theme"].astype(str).str.lower() + "__" + tr_raw["keyword_clean"]

trends = (
    tr_raw.pivot_table(index="Date", columns="colname", values="hits", aggfunc="mean")
          .sort_index()
          .reset_index()
)

# --- merge all macro + GT keyword data by Date
macro = reduce(lambda l,r: pd.merge(l, r, on="Date", how="outer"),
               [awe, unemp, cpi, appr, mort2, br, conf, const, trends])

# --- load HPI (explicit MDY)
hpi_raw = read_any(paths["HPI"]).copy()
if "Date" not in hpi_raw.columns:
    raise ValueError("HPI file must have a 'Date' column.")
hpi_raw["Date"] = to_month_mdy(hpi_raw["Date"])

# normalise identifiers
for c in list(hpi_raw.columns):
    cl = c.lower()
    if "region" in cl and "name" in cl and "RegionName" not in hpi_raw.columns:
        hpi_raw = hpi_raw.rename(columns={c:"RegionName"})
    if cl in ("areacode","area_code","code") and "AreaCode" not in hpi_raw.columns:
        hpi_raw = hpi_raw.rename(columns={c:"AreaCode"})

keep = [c for c in ["Date","RegionName","AreaCode","AveragePrice","Index","SalesVolume"] if c in hpi_raw.columns]
hpi   = hpi_raw[keep].sort_values(["RegionName","Date"] if "RegionName" in keep else "Date")

# --- clip to study window and merge
start, end = pd.Timestamp(2005,1,1), pd.Timestamp(2025,6,1)
macro = macro[(macro["Date"]>=start) & (macro["Date"]<=end)]
hpi   = hpi[(hpi["Date"]>=start) & (hpi["Date"]<=end)]

hpi_reg = pd.merge(hpi, macro, on="Date", how="left", validate="m:1")

# --- save with dd/mm/yyyy formatting
hpi_reg = hpi_reg.copy()
hpi_reg["Date"] = pd.to_datetime(hpi_reg["Date"])

xlsx_path = OUT / "HPI_regional_merged_2005_2025_ddmmyyyy_MODIFIED_with_GTkeywords.xlsx"
with pd.ExcelWriter(xlsx_path, engine="xlsxwriter", datetime_format="dd/mm/yyyy") as xl:
    hpi_reg.to_excel(xl, index=False, sheet_name="HPI_Regional")
    ws = xl.sheets["HPI_Regional"]; wb = xl.book
    ws.set_column(hpi_reg.columns.get_loc("Date"), hpi_reg.columns.get_loc("Date"), 12,
                  wb.add_format({"num_format":"dd/mm/yyyy"}))

csv_path = OUT / "HPI_regional_merged_2005_2025_ddmmyyyy_MODIFIED_with_GTkeywords.csv"
tmp = hpi_reg.copy(); tmp["Date"] = tmp["Date"].dt.strftime("%d/%m/%Y"); tmp.to_csv(csv_path, index=False)

print("Saved Excel:", xlsx_path)
print("Saved CSV  :", csv_path)
files.download(str(xlsx_path))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved Excel: /content/drive/MyDrive/Datasets/MERGED_OUTPUTS/HPI_regional_merged_2005_2025_ddmmyyyy_MODIFIED_with_GTkeywords.xlsx
Saved CSV  : /content/drive/MyDrive/Datasets/MERGED_OUTPUTS/HPI_regional_merged_2005_2025_ddmmyyyy_MODIFIED_with_GTkeywords.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>